In [69]:
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [70]:
X = tf.placeholder(tf.float32 , (None , 3), name ="X")

In [71]:
def numpy_use(x) :
    result = x- np.mean(x)
    return result
def sklearn_use(x) :
    result = minmax.transform(x)
    return result
def sklearn_f_use(x, func) :
    result = func.transform(x)
    return result
def sklearn_f_use(func) :
    def some_func(x) :
        result = func.transform(x)
        return result
    return some_func
def slice_f(x) :
    result = x[:,[0,1]]
    return result
x = np.random.normal(size = (10,3))
minmax = MinMaxScaler(feature_range=(-1,1))
minmax.fit(x)

MinMaxScaler(copy=True, feature_range=(-1, 1))

In [72]:
output = tf.py_func(numpy_use , [X], [tf.float32])[0]
sklearn_output = tf.py_func(sklearn_use , [X], [tf.float32])[0]
sklearn_f_output = tf.py_func(sklearn_f_use(minmax) , [X], [tf.float32])[0]
slice_output = tf.py_func(slice_f , [X], [tf.float32])[0]

In [75]:
output.set_shape((None , X.get_shape()[1]))
output

<tf.Tensor 'PyFunc:0' shape=(?, 3) dtype=float32>

In [70]:
sess = tf.InteractiveSession()

/root/anaconda3/lib/python3.6/site-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [71]:
sess.run(output, feed_dict={X:x})

array([[ 1.8857725 ,  1.9629173 , -1.6189413 ],
       [-0.9069593 ,  0.98138773,  0.34408993],
       [-0.8913342 , -1.4837158 , -0.7045703 ],
       [-0.54597664,  0.1585199 ,  0.6340972 ],
       [-0.05180714,  0.14778012,  0.69791985],
       [-0.0888017 , -1.4104047 , -0.6304159 ],
       [-0.07513703,  0.22047132, -1.292985  ],
       [ 0.51942   , -0.0026746 ,  1.3610324 ],
       [ 0.31763238,  0.7182206 ,  1.3381673 ],
       [ 0.6085986 , -1.445794  , -0.74650884]], dtype=float32)

In [72]:
sess.run(sklearn_output, feed_dict={X:x})

array([[ 1.        ,  0.99999994, -1.        ],
       [-1.        ,  0.43044147,  0.3174823 ],
       [-0.9888102 , -0.99999994, -0.38632277],
       [-0.7414841 , -0.04704933,  0.5121198 ],
       [-0.38758728, -0.05328137,  0.5549542 ],
       [-0.41408074, -0.9574593 , -0.33655426],
       [-0.40429488, -0.01110039, -0.7812354 ],
       [ 0.02149399, -0.14058669,  1.        ],
       [-0.12301515,  0.27773175,  0.98465425],
       [ 0.08535875, -0.97799486, -0.4144697 ]], dtype=float32)

In [73]:
sess.run(sklearn_f_output, feed_dict={X:x})

array([[ 1.        ,  0.99999994, -1.        ],
       [-1.        ,  0.43044147,  0.3174823 ],
       [-0.9888102 , -0.99999994, -0.38632277],
       [-0.7414841 , -0.04704933,  0.5121198 ],
       [-0.38758728, -0.05328137,  0.5549542 ],
       [-0.41408074, -0.9574593 , -0.33655426],
       [-0.40429488, -0.01110039, -0.7812354 ],
       [ 0.02149399, -0.14058669,  1.        ],
       [-0.12301515,  0.27773175,  0.98465425],
       [ 0.08535875, -0.97799486, -0.4144697 ]], dtype=float32)

In [74]:
sess.run(slice_output, feed_dict={X:x})

array([[ 2.0116918 ,  2.0888367 ],
       [-0.78103995,  1.1073071 ],
       [-0.76541483, -1.3577964 ],
       [-0.42005724,  0.28443927],
       [ 0.07411223,  0.2736995 ],
       [ 0.03711767, -1.2844853 ],
       [ 0.05078234,  0.3463907 ],
       [ 0.64533937,  0.12324477],
       [ 0.44355175,  0.84413993],
       [ 0.73451793, -1.3198746 ]], dtype=float32)

In [14]:
x = np.random.normal(size = (1000,2))
y = 5*x[:,[0]] +0.5*x[:,[1]] + 0.5

In [15]:
minmax = MinMaxScaler(feature_range=(-1,1))
minmax.fit(x)
y_minmax = MinMaxScaler(feature_range=(-1,1))
y_minmax.fit(y)
def sklearn_use(x) :
    result = minmax.transform(x)
    return result
def target_sklearn_use(y) :
    result = y_minmax.transform(y)
    return result

In [16]:
tf.reset_default_graph()
X = tf.placeholder(tf.float32 , (None , 2), name ="X")
Y = tf.placeholder(tf.float32 , (None , 1), name ="y")
w = tf.get_variable("w",[2,1])
b = tf.get_variable("b",[1])

In [17]:
data_tuple = (X,Y)
dataset = tf.data.Dataset.from_tensor_slices(data_tuple)
dataset = dataset.shuffle(buffer_size=100)
dataset = dataset.batch(batch_size=10, drop_remainder=True)
iter = dataset.make_initializable_iterator()
feature_batch , y_batch = iter.get_next()

In [18]:
transX = tf.py_func(sklearn_use, [feature_batch], [tf.float32])[0]
transY = tf.py_func(target_sklearn_use, [y_batch], [tf.float32])[0]

In [19]:
logit = tf.nn.bias_add(tf.matmul(transX,w),b)

In [20]:
mse = tf.reduce_mean(tf.square(tf.subtract(transY,logit)))

In [21]:
t_vars = tf.trainable_variables()

In [22]:
opt = tf.train.AdamOptimizer(1e-3).minimize(mse)

In [23]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
epoch_n = 1000
for i in range(epoch_n) :
    sess.run(iter.initializer , feed_dict= {X : x,
                                            Y : y
                                           }) # switch to train dataset
    while True :
        try :
            result = sess.run([opt,mse,w,b])
        except tf.errors.OutOfRangeError :
                break
    print(f"loss : {result[-3]} , w : {result[-2]} , b : {result[-1]}")


loss : 0.7469775676727295 , w : [[0.97521424]
 [0.69938713]] , b : [0.7466556]
loss : 0.5356089472770691 , w : [[0.949226  ]
 [0.62336606]] , b : [0.6544904]
loss : 0.3945908546447754 , w : [[0.9289799]
 [0.5517426]] , b : [0.5683942]
loss : 0.2978021800518036 , w : [[0.9112637]
 [0.4855729]] , b : [0.48851317]
loss : 0.2362479269504547 , w : [[0.90251565]
 [0.42511484]] , b : [0.41489252]
loss : 0.18014058470726013 , w : [[0.89514965]
 [0.3701265 ]] , b : [0.34766734]
loss : 0.12564018368721008 , w : [[0.8947168]
 [0.3214559]] , b : [0.28684327]
loss : 0.11017699539661407 , w : [[0.89574975]
 [0.2776378 ]] , b : [0.2323636]
loss : 0.05304364487528801 , w : [[0.90118384]
 [0.23959553]] , b : [0.18411031]
loss : 0.03368821367621422 , w : [[0.90839034]
 [0.20766273]] , b : [0.14199859]
loss : 0.019232671707868576 , w : [[0.918334  ]
 [0.18020698]] , b : [0.10561281]
loss : 0.013438333757221699 , w : [[0.9284245 ]
 [0.15772137]] , b : [0.07480343]
loss : 0.010957007296383381 , w : [[0.939

loss : 4.145642247592668e-15 , w : [[1.0251215 ]
 [0.10822747]] , b : [-0.04444938]
loss : 3.4819368762773525e-15 , w : [[1.0251215 ]
 [0.10822746]] , b : [-0.04444937]
loss : 1.4988010832439613e-15 , w : [[1.0251215 ]
 [0.10822745]] , b : [-0.04444938]
loss : 1.7881530437401875e-15 , w : [[1.0251215 ]
 [0.10822745]] , b : [-0.04444937]
loss : 1.4628055711174426e-15 , w : [[1.0251215 ]
 [0.10822747]] , b : [-0.04444937]
loss : 4.159520035400483e-15 , w : [[1.0251215 ]
 [0.10822745]] , b : [-0.04444937]
loss : 2.61249366070019e-16 , w : [[1.0251216 ]
 [0.10822748]] , b : [-0.04444937]
loss : 8.483664947506338e-16 , w : [[1.0251216 ]
 [0.10822746]] , b : [-0.04444939]
loss : 8.420564698705037e-16 , w : [[1.0251216 ]
 [0.10822745]] , b : [-0.04444938]
loss : 6.526029928382985e-16 , w : [[1.0251216 ]
 [0.10822745]] , b : [-0.04444938]
loss : 2.970713952610282e-16 , w : [[1.0251216 ]
 [0.10822745]] , b : [-0.04444938]
loss : 1.7433970933566911e-15 , w : [[1.0251216 ]
 [0.10822745]] , b : [-

loss : 3.954299110375814e-10 , w : [[1.0250498 ]
 [0.10826918]] , b : [-0.0444627]
loss : 4.374812957643609e-13 , w : [[1.0251199 ]
 [0.10822675]] , b : [-0.04444858]
loss : 1.2854604194783258e-14 , w : [[1.0251215 ]
 [0.10822716]] , b : [-0.0444494]
loss : 9.271229829116282e-16 , w : [[1.0251216 ]
 [0.10822745]] , b : [-0.04444936]
loss : 1.9464984840427378e-14 , w : [[1.0251218 ]
 [0.10822752]] , b : [-0.0444493]
loss : 3.1488699688898055e-15 , w : [[1.0251216 ]
 [0.10822758]] , b : [-0.04444943]
loss : 7.639722188201858e-15 , w : [[1.0251218 ]
 [0.10822743]] , b : [-0.04444941]
loss : 1.2267964633866217e-15 , w : [[1.0251217 ]
 [0.10822739]] , b : [-0.04444936]
loss : 2.1222693402460878e-14 , w : [[1.0251216 ]
 [0.10822789]] , b : [-0.04444937]
loss : 2.0539126220263192e-16 , w : [[1.0251216 ]
 [0.10822748]] , b : [-0.04444938]
loss : 2.3425706243107277e-15 , w : [[1.0251217 ]
 [0.10822732]] , b : [-0.04444937]
loss : 1.253005538952648e-13 , w : [[1.0251211]
 [0.1082278]] , b : [-0.

loss : 1.029390635232108e-11 , w : [[1.0251257 ]
 [0.10821971]] , b : [-0.04445143]
loss : 2.645081043084474e-09 , w : [[1.0251776 ]
 [0.10810174]] , b : [-0.04439914]
loss : 3.087691027303663e-07 , w : [[1.0244043 ]
 [0.10822164]] , b : [-0.04394741]
loss : 8.205313650755386e-10 , w : [[1.0252261 ]
 [0.10811478]] , b : [-0.04442956]
loss : 1.0518432652650844e-10 , w : [[1.0251161 ]
 [0.10820892]] , b : [-0.04445587]
loss : 1.1989064255257809e-13 , w : [[1.0251198]
 [0.1082266]] , b : [-0.04444939]
loss : 1.7079242005586283e-14 , w : [[1.0251217 ]
 [0.10822784]] , b : [-0.04444947]
loss : 1.1669918503764976e-14 , w : [[1.0251215 ]
 [0.10822751]] , b : [-0.0444495]
loss : 4.174438403183999e-15 , w : [[1.0251216 ]
 [0.10822743]] , b : [-0.04444936]
loss : 3.5701693337775176e-15 , w : [[1.0251213 ]
 [0.10822759]] , b : [-0.04444942]
loss : 1.0987356993197653e-13 , w : [[1.0251215 ]
 [0.10822809]] , b : [-0.04444917]
loss : 7.613527863714609e-14 , w : [[1.0251222 ]
 [0.10822749]] , b : [-0

loss : 6.360544729799855e-11 , w : [[1.025127  ]
 [0.10824174]] , b : [-0.04445899]
loss : 5.5424566845763135e-12 , w : [[1.0251249 ]
 [0.10822345]] , b : [-0.04444972]
loss : 6.110523328647921e-13 , w : [[1.025121  ]
 [0.10822701]] , b : [-0.04444858]
loss : 1.3829660073377825e-12 , w : [[1.0251215 ]
 [0.10823046]] , b : [-0.04444988]
loss : 3.4102386713141808e-12 , w : [[1.0251197 ]
 [0.10822476]] , b : [-0.04445033]
loss : 2.9568251059403083e-12 , w : [[1.0251255 ]
 [0.10822617]] , b : [-0.04444794]
loss : 4.087828386900583e-08 , w : [[1.0251812 ]
 [0.10903799]] , b : [-0.04447853]
loss : 2.6248709872334075e-08 , w : [[1.025178  ]
 [0.10871422]] , b : [-0.0446263]
loss : 1.6114836132175014e-09 , w : [[1.0250957 ]
 [0.10833465]] , b : [-0.04447376]
loss : 1.2044646391373703e-09 , w : [[1.0252478 ]
 [0.10819149]] , b : [-0.04444072]
loss : 1.1674197719635515e-10 , w : [[1.0250987 ]
 [0.10819882]] , b : [-0.04444005]
loss : 1.0753972823351887e-08 , w : [[1.0250518 ]
 [0.10789689]] , b 

...

loss : 3.4155113633194123e-12 , w : [[1.0251296 ]
 [0.10823307]] , b : [-0.04444942]
loss : 3.3403940724596737e-10 , w : [[1.0251017 ]
 [0.10825376]] , b : [-0.04446369]
loss : 1.0279588025241537e-07 , w : [[1.0252331 ]
 [0.10851737]] , b : [-0.04475273]
loss : 3.753669819417382e-09 , w : [[1.0252804 ]
 [0.10809959]] , b : [-0.04440913]
loss : 5.555041315830067e-09 , w : [[1.0251054]
 [0.1079589]] , b : [-0.04447632]
loss : 3.767634260132269e-10 , w : [[1.0251205 ]
 [0.10824258]] , b : [-0.04446948]
loss : 4.618415871959769e-08 , w : [[1.025048  ]
 [0.10776019]] , b : [-0.0445061]
loss : 4.566953037965504e-09 , w : [[1.0250285 ]
 [0.10858109]] , b : [-0.04440025]
loss : 1.0734911626286703e-07 , w : [[1.0240923 ]
 [0.10892269]] , b : [-0.04460977]
loss : 1.5787079759785327e-12 , w : [[1.025122  ]
 [0.10823473]] , b : [-0.04445076]
loss : 9.608119421605776e-13 , w : [[1.0251216 ]
 [0.10822742]] , b : [-0.0444484]
loss : 1.5718821643612158e-12 , w : [[1.0251206 ]
 [0.10823086]] , b : [-0.

loss : 5.2263509076055925e-09 , w : [[1.0252073 ]
 [0.10825449]] , b : [-0.04451353]
loss : 6.88545256366524e-08 , w : [[1.0245034]
 [0.1079228]] , b : [-0.04440617]
loss : 6.330348867322755e-08 , w : [[1.0259246 ]
 [0.10761446]] , b : [-0.04440009]
loss : 2.1662616145334823e-11 , w : [[1.0251184 ]
 [0.10821053]] , b : [-0.04444744]
loss : 1.699768755584227e-15 , w : [[1.0251215 ]
 [0.10822768]] , b : [-0.04444942]
loss : 3.164135620181696e-15 , w : [[1.0251215 ]
 [0.10822759]] , b : [-0.04444939]
loss : 2.448388756345313e-15 , w : [[1.0251217 ]
 [0.10822739]] , b : [-0.04444934]
loss : 5.656933255160368e-15 , w : [[1.0251216 ]
 [0.10822755]] , b : [-0.04444947]
loss : 3.2971021746153184e-13 , w : [[1.0251204 ]
 [0.10822659]] , b : [-0.044449]
loss : 3.5065700766911652e-15 , w : [[1.0251213 ]
 [0.10822731]] , b : [-0.04444939]
loss : 7.1279787627887e-15 , w : [[1.0251215 ]
 [0.10822763]] , b : [-0.04444949]
loss : 3.9507287008912553e-14 , w : [[1.025121  ]
 [0.10822717]] , b : [-0.0444

loss : 2.1043063125336658e-13 , w : [[1.0251217 ]
 [0.10822665]] , b : [-0.04444893]
loss : 6.0517298637596806e-12 , w : [[1.0251259 ]
 [0.10823309]] , b : [-0.04445035]
loss : 2.250935438041779e-09 , w : [[1.0252126 ]
 [0.10810325]] , b : [-0.04445696]
loss : 4.5495995859567984e-11 , w : [[1.0251163 ]
 [0.10821451]] , b : [-0.0444512]
loss : 2.0921153698338912e-10 , w : [[1.0251076 ]
 [0.10819057]] , b : [-0.04443863]
loss : 5.874583735021588e-07 , w : [[1.0241469 ]
 [0.10723868]] , b : [-0.0447006]
loss : 6.8328214197777015e-09 , w : [[1.0254047 ]
 [0.10832781]] , b : [-0.04448771]
loss : 1.1952037137774596e-09 , w : [[1.0250785 ]
 [0.10815149]] , b : [-0.04442593]
loss : 2.857826475466396e-10 , w : [[1.0251372 ]
 [0.10822234]] , b : [-0.04443307]
loss : 2.7717498096518044e-14 , w : [[1.0251219 ]
 [0.10822767]] , b : [-0.04444931]
loss : 4.719558145444176e-14 , w : [[1.0251211 ]
 [0.10822754]] , b : [-0.04444948]
loss : 6.861178207480173e-15 , w : [[1.0251217 ]
 [0.10822728]] , b : [

loss : 4.9306739924705537e-14 , w : [[1.0251219 ]
 [0.10822778]] , b : [-0.04444933]
loss : 1.180888747247602e-13 , w : [[1.0251219 ]
 [0.10822766]] , b : [-0.04444904]
loss : 1.7542106547745184e-13 , w : [[1.0251199 ]
 [0.10822678]] , b : [-0.04444935]
loss : 4.331153559172969e-14 , w : [[1.0251222 ]
 [0.10822766]] , b : [-0.04444945]
loss : 8.387318888459366e-14 , w : [[1.0251211 ]
 [0.10822775]] , b : [-0.04444923]
loss : 2.2889648076257485e-12 , w : [[1.0251234]
 [0.108225 ]] , b : [-0.0444507]
loss : 3.2657376891620515e-07 , w : [[1.0240549 ]
 [0.10823741]] , b : [-0.04487119]
loss : 4.6448576540569775e-08 , w : [[1.0245435 ]
 [0.10865951]] , b : [-0.04439269]
loss : 1.2878580424313668e-09 , w : [[1.0250598 ]
 [0.10827836]] , b : [-0.04442606]
loss : 6.726285923640968e-14 , w : [[1.0251206 ]
 [0.10822749]] , b : [-0.04444921]
loss : 1.626476688724207e-15 , w : [[1.0251216]
 [0.1082274]] , b : [-0.04444939]
loss : 8.291718176054331e-15 , w : [[1.0251216 ]
 [0.10822742]] , b : [-0.0

loss : 2.7555673298707006e-09 , w : [[1.0251158 ]
 [0.10839279]] , b : [-0.04446691]
loss : 1.1965568091909518e-08 , w : [[1.0252613 ]
 [0.10846049]] , b : [-0.04440912]
loss : 3.931670633505746e-08 , w : [[1.0247401 ]
 [0.10805655]] , b : [-0.0445519]
loss : 9.053661154112547e-10 , w : [[1.0251045 ]
 [0.10831678]] , b : [-0.04446555]
loss : 6.590145096296851e-10 , w : [[1.0251285 ]
 [0.10819141]] , b : [-0.04446346]
loss : 2.3467982765423834e-11 , w : [[1.0251206 ]
 [0.10823604]] , b : [-0.04444791]
loss : 1.7879055874692185e-11 , w : [[1.0251101 ]
 [0.10822599]] , b : [-0.04444779]
loss : 1.4148971994032422e-09 , w : [[1.0251278 ]
 [0.10828365]] , b : [-0.04448725]
loss : 1.9526766315003385e-10 , w : [[1.0250934 ]
 [0.10824564]] , b : [-0.04445814]
loss : 5.550289117195462e-10 , w : [[1.025091  ]
 [0.10813382]] , b : [-0.04443905]
loss : 1.0390805993409913e-09 , w : [[1.0251946 ]
 [0.10828549]] , b : [-0.04445102]
loss : 1.2176033692412602e-07 , w : [[1.0243083 ]
 [0.10767543]] , b :

In [128]:
x = np.random.normal(size = (1000,2),loc = 5)
y = 5*x[:,[0]] +0.5*x[:,[1]] + 0.5

minmax = MinMaxScaler(feature_range=(-1,1))
minmax.fit(x)
y_minmax = MinMaxScaler(feature_range=(-1,1))
y_minmax.fit(y)

MinMaxScaler(copy=True, feature_range=(-1, 1))

In [225]:
def py_func(x, y):
    x_result = minmax.transform(x)
    y_result = y_minmax.transform(x)
    return x_result , y_result

tf.reset_default_graph()
X = tf.placeholder(tf.float32 , (None , 2), name ="X")
Y = tf.placeholder(tf.float32 , (None , 1), name ="y")
w = tf.get_variable("w",[2,1])
b = tf.get_variable("b",[1])

data_tuple = (X,Y)
dataset = tf.data.Dataset.from_tensor_slices(data_tuple)
dataset = dataset.map(
    lambda data_list, label_list: tuple(tf.py_func(py_func, 
                                                   [X, Y], 
                                                   [tf.float32, tf.float32])))
dataset = dataset.shuffle(buffer_size=100)
dataset = dataset.batch(batch_size=10, drop_remainder=True)
iter = dataset.make_initializable_iterator()
feature_batch , y_batch = iter.get_next()

In [207]:
# def py_func(x, y):
#     x = pd.DataFrame(x,columns=["x1","x2"])
#     print(x)
#     x_result = minmax.transform(x.values)
#     y_result = y_minmax.transform(y)
#     return x_result , y_result
# feature_batch , y_batch = tf.py_func(py_func,[feature_batch,y_batch],[tf.float32 , tf.float32])

In [208]:
feature_batch.set_shape([None,2])
y_batch.set_shape([None,1])

In [ ]:
feature_batch

In [209]:
# feature_batch = tf.reshape(feature_batch,[-1,X.get_shape()[1]])
# y_batch = tf.reshape(y_batch,[-1,Y.get_shape()[1]])

In [210]:
logit = tf.nn.bias_add(tf.matmul(feature_batch,w),b)
mse = tf.reduce_mean(tf.square(tf.subtract(y_batch,logit)))
opt = tf.train.AdamOptimizer(1e-3).minimize(mse)

In [211]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
epoch_n = 1000
for i in range(epoch_n) :
    sess.run(iter.initializer , feed_dict= {X : x,
                                            Y : y
                                           }) # switch to train dataset
    while True :
        try :
            result = sess.run([opt,mse,w,b, feature_batch])
        except tf.errors.OutOfRangeError :
                break
    print(f"loss : {result[1]} , w : {result[2]} , b : {result[3]}")

         x1        x2
0  4.927638  6.192213
1  5.514686  6.068861
2  5.829319  5.771373
3  6.533607  5.230089
4  4.951509  5.245216
5  4.994502  4.994112
6  4.947938  5.482284
7  7.007102  6.325680
8  4.687064  5.800726
9  4.137389  4.258953
         x1        x2
0  7.004644  6.007151
1  4.201903  5.585000
2  5.470720  3.769989
3  3.922042  4.067575
4  3.520906  5.970317
5  5.564907  4.503846
6  5.110385  5.917025
7  3.776581  3.386174
8  4.207416  5.318051
9  5.220351  5.759227
         x1        x2
0  5.605479  6.277387
1  4.992818  5.035080
2  4.242191  6.209707
3  4.381508  5.406266
4  4.799785  5.994912
5  5.374750  6.740628
6  4.437182  6.338117
7  5.270819  5.324787
8  4.725610  3.706900
9  6.113008  4.123618
         x1        x2
0  5.782512  6.008763
1  5.168511  4.868842
2  3.940382  6.761811
3  4.996192  5.981555
4  3.802149  5.451237
5  6.382689  5.149343
6  4.295438  5.279954
7  7.299850  6.106343
8  5.618940  4.683245
9  6.363926  4.646717
         x1        x2
0  5.35328

         x1        x2
0  3.121697  4.950528
1  3.791584  5.527342
2  4.568502  4.433037
3  5.136581  4.390023
4  6.222874  5.892486
5  5.544522  6.774680
6  2.927110  5.409407
7  2.309752  4.404546
8  3.784014  4.671433
9  4.302162  6.385056
         x1        x2
0  5.684203  4.715843
1  5.556709  5.397525
2  3.973426  4.710943
3  5.768041  4.716628
4  4.995323  3.860077
5  4.844131  5.383495
6  4.314550  4.789917
7  2.790422  5.176113
8  5.354246  6.501931
9  4.782992  3.153874
         x1        x2
0  4.686459  5.963547
1  4.487861  5.214761
2  4.662531  4.382164
3  6.004234  6.031823
4  5.195746  6.260084
5  5.725501  7.355972
6  3.350727  5.462876
7  4.781380  5.375377
8  7.100265  4.305667
9  3.457847  4.607715
         x1        x2
0  4.471231  4.978420
1  5.950418  3.846953
2  4.953836  4.876764
3  4.802753  2.670359
4  2.821515  6.401792
5  4.307066  3.642627
6  4.249775  6.937878
7  5.482639  4.539541
8  5.260655  3.744127
9  6.001198  7.210035
         x1        x2
0  4.58012

         x1        x2
0  4.307714  3.377707
1  5.039053  4.720773
2  4.402461  4.036509
3  4.948856  5.841919
4  5.681319  5.660011
5  4.222168  3.606097
6  5.254871  3.976364
7  4.447050  5.999466
8  4.513944  6.460198
9  4.829028  3.422848
         x1        x2
0  3.695987  5.343976
1  4.889887  4.301159
2  4.012985  5.431122
3  5.175100  4.785471
4  4.030788  4.058696
5  5.594982  4.858512
6  5.860738  2.905558
7  4.082505  7.199149
8  6.032058  3.858443
9  3.329448  6.332291
         x1        x2
0  5.591146  4.105499
1  4.314964  3.350923
2  3.580933  5.347291
3  4.464242  3.762413
4  5.385030  6.657021
5  4.224723  5.080879
6  3.449253  5.405969
7  6.081844  4.580251
8  5.392653  4.525473
9  3.982009  3.341851
         x1        x2
0  7.714210  5.877428
1  5.321304  3.691823
2  4.021969  4.603218
3  6.070019  5.121380
4  4.866362  5.596677
5  6.059806  4.631524
6  4.412294  4.201920
7  3.744398  6.275367
8  4.882649  5.958279
9  5.210783  4.935067
         x1        x2
0  2.99772

         x1        x2
0  5.584875  5.445014
1  4.977198  6.204465
2  5.931054  4.529220
3  4.676947  3.284312
4  6.641151  5.174326
5  5.773832  4.213409
6  5.312535  6.640288
7  5.829319  5.771373
8  5.119278  5.783016
9  7.009934  5.667230
         x1        x2
0  4.242910  5.145285
1  4.671724  4.755598
2  2.872530  4.281612
3  5.500242  5.466198
4  6.003939  5.171796
5  4.412446  6.455375
6  4.304651  3.883599
7  5.129546  5.380898
8  3.093748  4.839868
9  5.154072  4.676261
         x1        x2
0  4.996192  5.981555
1  4.343232  5.186404
2  4.689354  5.345148
3  4.606666  4.076204
4  3.767410  6.045959
5  3.896276  4.353718
6  5.608321  2.695600
7  4.746368  4.494372
8  5.634499  5.555451
9  4.146727  3.044817
         x1        x2
0  5.034573  5.166427
1  4.015775  4.749055
2  5.223372  4.786848
3  3.415883  6.835860
4  3.666528  5.449451
5  5.011476  3.320433
6  4.381508  5.406266
7  3.693759  4.261216
8  6.871408  6.005315
9  5.965245  5.820779
         x1        x2
0  4.78138

         x1        x2
0  5.671493  5.793046
1  5.455298  7.450051
2  6.144930  3.492933
3  4.122494  6.854495
4  4.951046  5.892739
5  5.098315  5.531209
6  5.635932  4.601449
7  4.580047  4.670702
8  4.534390  5.032629
9  5.760293  6.276021
         x1        x2
0  5.422648  4.377346
1  5.704873  4.208464
2  4.097610  3.794938
3  5.302448  3.529922
4  2.883836  6.382099
5  5.093496  4.396631
6  4.403279  3.240859
7  5.795355  6.143123
8  4.897070  5.677280
9  6.726755  5.277539
         x1        x2
0  6.044723  4.803416
1  6.901021  4.979663
2  5.457621  5.148341
3  4.349268  5.217976
4  6.144146  5.668527
5  4.513944  6.460198
6  5.544522  6.774680
7  4.322018  5.000241
8  4.547539  5.575861
9  5.236314  6.362502
         x1        x2
0  6.559212  4.122263
1  2.839445  5.844032
2  4.988757  4.267790
3  5.515097  5.164618
4  5.714615  5.431245
5  6.331804  5.066294
6  6.016535  3.232972
7  7.296916  5.485002
8  4.515443  3.885927
9  6.192101  4.726191
         x1        x2
0  6.51444

...

         x1        x2
0  4.781716  5.493623
1  5.372090  5.020515
2  5.269253  6.020608
3  5.787933  3.556901
4  6.119947  5.919783
5  4.749214  4.721930
6  5.629856  3.618096
7  4.341275  5.706986
8  5.988122  3.231919
9  5.042757  3.782116
         x1        x2
0  7.296916  5.485002
1  3.435671  4.369507
2  5.501192  5.127851
3  5.501987  5.460707
4  4.334116  5.781462
5  5.212900  5.564235
6  3.381387  6.253203
7  4.059649  3.686170
8  5.355929  3.569228
9  5.043469  4.428694
         x1        x2
0  3.499037  4.726599
1  6.267111  5.138373
2  6.050700  4.830063
3  4.977738  5.130785
4  4.444526  4.064501
5  6.514449  4.783582
6  5.641660  5.480633
7  5.095620  5.936924
8  6.682633  4.921834
9  4.752496  4.752845
         x1        x2
0  5.302448  3.529922
1  5.793555  6.104155
2  5.455298  7.450051
3  5.482639  4.539541
4  6.070889  5.910651
5  4.236800  5.453459
6  5.013597  5.056028
7  6.254343  4.361429
8  2.599108  5.862816
9  5.459671  5.998731
         x1        x2
0  6.30030

         x1        x2
0  4.310151  5.995810
1  5.096007  4.881512
2  3.098094  5.033266
3  4.239225  6.335523
4  3.926188  4.480130
5  5.386427  6.248710
6  7.341582  4.251737
7  2.650098  7.354864
8  7.714210  5.877428
9  4.314964  3.350923
         x1        x2
0  4.965749  3.735293
1  5.577331  6.822793
2  6.282741  5.509853
3  5.214209  4.208049
4  3.930536  4.110681
5  5.175100  4.785471
6  5.707981  5.381863
7  5.078817  5.366581
8  4.554281  4.638218
9  3.625870  4.424179
         x1        x2
0  6.208778  4.610519
1  4.012985  5.431122
2  3.592805  5.132964
3  4.189792  5.639828
4  5.637291  5.772589
5  4.512906  5.284764
6  3.679691  4.045610
7  3.268073  4.540646
8  7.752546  5.482546
9  5.594982  4.858512
         x1        x2
0  5.691991  5.872889
1  3.329469  5.348525
2  4.486239  3.306565
3  3.986170  3.543334
4  4.460895  4.411871
5  4.829028  3.422848
6  5.687690  5.331684
7  5.979165  4.043918
8  4.637464  3.119952
9  3.486338  5.245565
         x1        x2
0  6.61822

         x1        x2
0  5.080027  3.934180
1  3.742167  5.971562
2  4.388273  6.210017
3  6.755771  5.658813
4  5.260655  3.744127
5  6.199178  5.955550
6  1.792226  2.581118
7  4.423383  5.920013
8  5.784781  5.719920
9  4.984429  3.810056
         x1        x2
0  4.109196  4.871730
1  6.425744  5.231041
2  5.929699  3.572087
3  6.277466  4.246703
4  6.687785  4.557696
5  5.451098  5.725218
6  4.281927  4.058170
7  5.117271  3.846881
8  4.383544  3.569017
9  4.652674  3.861214
         x1        x2
0  5.364257  6.611204
1  4.175622  5.070948
2  4.844131  5.383495
3  4.812533  6.515093
4  2.957705  5.132154
5  6.534974  5.488785
6  5.543608  5.050245
7  5.203122  4.280576
8  6.531865  5.076219
9  2.821515  6.401792
         x1        x2
0  4.796168  6.735122
1  4.663641  6.007920
2  5.195746  6.260084
3  4.687064  5.800726
4  3.895744  4.300791
5  3.195863  4.508531
6  6.156522  3.165089
7  5.789713  4.591001
8  3.874196  4.039317
9  3.882038  5.820199
         x1        x2
0  5.82200

         x1        x2
0  4.597135  4.806507
1  6.027893  5.701418
2  3.758427  4.032007
3  4.456752  5.889250
4  3.004873  5.129152
5  6.618223  5.091883
6  4.700228  3.023033
7  4.954862  5.929388
8  4.412162  3.125912
9  7.117881  4.609448
         x1        x2
0  4.139184  4.854989
1  4.549517  4.647350
2  5.904737  5.587206
3  3.608429  3.628901
4  3.965546  6.358589
5  4.789723  7.137992
6  5.833513  4.391432
7  6.446151  4.605334
8  4.534390  5.032629
9  5.130481  5.271006
         x1        x2
0  6.144146  5.668527
1  5.301348  5.372598
2  5.439062  6.923954
3  2.622893  4.173805
4  5.354246  6.501931
5  5.890339  6.243935
6  3.898409  3.811433
7  4.053957  6.495976
8  4.821098  5.272721
9  3.329448  6.332291
         x1        x2
0  5.460259  3.024904
1  4.377330  4.296180
2  4.804446  4.759594
3  5.096007  4.881512
4  4.575607  5.507348
5  6.446684  4.772575
6  5.970923  4.665958
7  4.684550  5.290620
8  6.032058  3.858443
9  3.940235  3.570014
         x1        x2
0  3.81826

KeyboardInterrupt: 

In [205]:
result[-1].shape

(10, 2)

In [240]:
x = np.random.normal(size = (1000,2),loc = 5)
y = 5*x[:,[0]] +0.5*x[:,[1]] + 0.5
minmax = MinMaxScaler(feature_range=(-1,1))
minmax.fit(x)
y_minmax = MinMaxScaler(feature_range=(-1,1))
y_minmax.fit(y)

def py_func(x, y):
    print(x)
    print(y)
    x_result = minmax.transform(x)
    y_result = y_minmax.transform(y)
    return x_result , y_result

tf.reset_default_graph()
X = tf.placeholder(tf.float32 , (None , 2), name ="X")
Y = tf.placeholder(tf.float32 , (None , 1), name ="y")
w = tf.get_variable("w",[2,1])
b = tf.get_variable("b",[1])
data_tuple = (X,Y)
dataset = tf.data.Dataset.from_tensor_slices(data_tuple)
dataset = dataset.map(
    lambda x, y: tuple(tf.py_func(py_func, [x, y], 
                                  [tf.float32, tf.float32])))
# dataset = dataset.shuffle(buffer_size=100)
dataset = dataset.batch(batch_size=10, drop_remainder=True)
iter = dataset.make_initializable_iterator()
feature_batch , y_batch = iter.get_next()

In [241]:
# feature_batch.set_shape([None,2])
# y_batch.set_shape([None,1])

In [242]:
logit = tf.nn.bias_add(tf.matmul(feature_batch,w),b)
mse = tf.reduce_mean(tf.square(tf.subtract(y_batch,logit)))
opt = tf.train.AdamOptimizer(1e-3).minimize(mse)

In [243]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
epoch_n = 1000
for i in range(epoch_n) :
    sess.run(iter.initializer , feed_dict= {X : x,
                                            Y : y
                                           }) # switch to train dataset
    while True :
        try :
            result = sess.run([opt,mse,w,b, feature_batch])
        except tf.errors.OutOfRangeError :
                break
    print(f"loss : {result[1]} , w : {result[2]} , b : {result[3]}, x shape : {result[-1].shape}")

[5.0422473 4.90878  ]
[28.165627]
[4.8235044 4.5335865]
[26.884315]


InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument: {{function_node __inference_Dataset_map_<lambda>_5256}} ValueError: Expected 2D array, got 1D array instead:
array=[5.0422473 4.90878  ].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.
Traceback (most recent call last):

  File "/root/anaconda3/lib/python3.6/site-packages/tensorflow_core/python/ops/script_ops.py", line 235, in __call__
    ret = func(*args)

  File "<ipython-input-240-8bca171c1f15>", line 11, in py_func
    x_result = minmax.transform(x)

  File "/root/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_data.py", line 412, in transform
    force_all_finite="allow-nan")

  File "/root/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py", line 556, in check_array
    "if it contains a single sample.".format(array))

ValueError: Expected 2D array, got 1D array instead:
array=[5.0422473 4.

In [125]:
result[-1][3,:,:]

array([[4.732565 , 4.173401 ],
       [3.921896 , 4.387159 ],
       [4.96026  , 6.105776 ],
       ...,
       [5.753937 , 4.832196 ],
       [4.2840242, 5.386073 ],
       [4.3995585, 4.2459106]], dtype=float32)

In [260]:
x = np.random.normal(size = (1000,2),loc = 5)
y = 5*x[:,[0]] +0.5*x[:,[1]] + 0.5

minmax = MinMaxScaler(feature_range=(-1,1))
minmax.fit(x)
y_minmax = MinMaxScaler(feature_range=(-1,1))
y_minmax.fit(y)

def py_func(x, y):
    x_result = minmax.transform(x)
    y_result = y_minmax.transform(y)
    return x_result , y_result

tf.reset_default_graph()
X = tf.placeholder(tf.float32 , (None , 2), name ="X")
Y = tf.placeholder(tf.float32 , (None , 1), name ="y")
w = tf.get_variable("w",[2,1])
b = tf.get_variable("b",[1])

data_tuple = (X,Y)
dataset = tf.data.Dataset.from_tensor_slices(data_tuple)
dataset = dataset.shuffle(buffer_size=100)
dataset = dataset.batch(batch_size=10, drop_remainder=True)
dataset = dataset.map(
    lambda xx, yy: tuple(tf.py_func(py_func, [xx, yy], [tf.float32, tf.float32])))

iter = dataset.make_initializable_iterator()
feature_batch , y_batch = iter.get_next()

feature_batch.set_shape([None,2])
y_batch.set_shape([None,1])

logit = tf.nn.bias_add(tf.matmul(feature_batch,w),b)
mse = tf.reduce_mean(tf.square(tf.subtract(y_batch,logit)))
opt = tf.train.AdamOptimizer(1e-3).minimize(mse)

sess = tf.Session()
sess.run(tf.global_variables_initializer())
epoch_n = 1000
for i in range(epoch_n) :
    sess.run(iter.initializer , feed_dict= {X : x,
                                            Y : y
                                           }) # switch to train dataset
    while True :
        try :
            result = sess.run([opt,mse,w,b, feature_batch])
        except tf.errors.OutOfRangeError :
                break
    print(f"loss : {result[1]} , w : {result[2]} , b : {result[3]}")

loss : 0.6032754778862 , w : [[0.74111265]
 [1.3368484 ]] , b : [-0.6520206]
loss : 0.4006151258945465 , w : [[0.810486 ]
 [1.2645264]] , b : [-0.56263024]
loss : 0.5159962773323059 , w : [[0.8710924]
 [1.19261  ]] , b : [-0.48105785]
loss : 0.3175644278526306 , w : [[0.9235347]
 [1.1182996]] , b : [-0.40582696]
loss : 0.22507746517658234 , w : [[0.9670002]
 [1.047929 ]] , b : [-0.33795178]
loss : 0.20973224937915802 , w : [[1.0034673 ]
 [0.97859794]] , b : [-0.27776715]
loss : 0.18888625502586365 , w : [[1.0315   ]
 [0.9111758]] , b : [-0.22439992]
loss : 0.11866903305053711 , w : [[1.0522121 ]
 [0.84605473]] , b : [-0.17866914]
loss : 0.029368912801146507 , w : [[1.0668942 ]
 [0.78310925]] , b : [-0.13909067]
loss : 0.03521033376455307 , w : [[1.0755321]
 [0.7226405]] , b : [-0.10593147]
loss : 0.024282287806272507 , w : [[1.0779895 ]
 [0.66469204]] , b : [-0.07838632]
loss : 0.023257557302713394 , w : [[1.0773023 ]
 [0.60987085]] , b : [-0.05572639]
loss : 0.014215292409062386 , w :

KeyboardInterrupt: 

In [261]:
result[-1].shape

(10, 2)